# Self Healing Glass

In [ ]:
from lammps import PyLammps

In [ ]:
L = PyLammps()

#### SET SIMULATION CONDITIONS

In [ ]:
L.command("units         lj")
L.command("atom_style    full")
L.command("dimension     3")
L.command("boundary p p p")
L.command("neighbor 0.8 bin")

L.command("bond_style    harmonic")
L.command("pair_style lj/expand 2.5")

In [ ]:
L.command("read_data fracturedGlass.data")

#### POTENTIALS

In [ ]:
L.command("bond_coeff    * 2500.0 0.9")
L.command("special_bonds fene")
L.command("pair_coeff * * 1.0 1.0 0.0 2.5")

#### VARIABLES DEFINITION

In [ ]:
L.command("variable Temp equal 0.1")
L.command("variable dt equal 0.005")
L.command("variable Tdamping equal 100*${dt}")

#### THERMO-STYLE + INTEGRATION

In [ ]:
L.command("thermo  200")
L.command("thermo_style    custom step temp ebond epair ke etotal enthalpy vol press")
L.command("velocity     all create ${Temp} 12398412")
L.command("timestep     ${dt}")

#### PRODUCTION

In [ ]:
L.command("fix nvtProd all nvt temp ${Temp} ${Temp} ${Tdamping}")
L.command("dump mydump all custom 200 Conf.dat id type xu yu zu vx vy vz")
L.command("dump_modify mydump sort id")

L.command("run 3000")

L.command("undump mydump")
L.command("unfix nvtProd")
L.command("reset_timestep 0")

#### OSCILLATIONS

In [ ]:
L.command("change_box     all triclinic")

L.command("variable tmp equal lx")
L.command("variable L0 equal ${tmp}")
L.command("variable Amp equal lx*0.15  ") #% of box length
L.command("variable Tp equal 10        ") #1 LJ time = 1ps, Tp=1000 --> 10^12/1000 = 10^9 Hz
L.command("variable runTime equal 2.5*${Tp}/${dt}")

L.command("compute         1 all pressure thermo_temp")
L.command("compute         usual all temp")
L.command("compute         tilt all temp/deform")
L.command("variable        strain equal xy/v_L0")
L.command("fix             strainData all ave/time 1 1 1000 c_1[1] c_1[2] c_1[3] c_1[4] c_1[5] c_1[6] file stress.data")
L.command("thermo_style     custom step temp c_usual density press etotal pxy v_strain")
L.command("thermo_modify   temp tilt")

L.command("fix     nvt1 all nvt/sllod temp ${Temp} ${Temp} ${Tdamping}")
L.command("fix     oscillate all deform 1 xy wiggle ${Amp} ${Tp} units box remap v")

L.command("dump mydumpOsc all custom 200 ConfOsc.dat id type xu yu zu vx vy vz")
L.command("dump_modify mydumpOsc sort id")

L.command("run      ${runTime}")

In [ ]:
import nglview as nv
import MDAnalysis as mda

In [ ]:
u = mda.Universe('Conf.dat',format='LAMMPSDUMP')
u_osc = mda.Universe('ConfOsc.dat',format='LAMMPSDUMP')
atoms = u.atoms
atoms_osc = u_osc.atoms

In [ ]:
view = nv.show_mdanalysis(atoms)
view_osc = nv.show_mdanalysis(atoms_osc)

In [ ]:
view

In [ ]:
view_osc